## Mónadas

Descripción componibles de computaciones

Permiten suplementar las funcionalidades puras con I/O, estado, indeterminismo, etc.

En terminos del lenguaje una mónada es un tipo instancia de la clase *Monad*.

```Haskell
class Monad m where
    return :: a -> m a
    (>>=) :: m a -> ( a -> m b ) -> m b
    (>>) :: m a -> m b -> m b
```

Mónada se puede ver como un contenedor: *return* pone el valor **a** adentro de ella.

Operador *bind* aplica una función al contenido de la mónada devolviendo otra mónada

In [1]:
import Control.Applicative

putStrLn "Como te llamas?"
    >>= (\_ -> getLine)
    >>= (\name -> putStrLn ("Hola, " ++ name ++ "!"))

Line 2: Use const
Found:
\ _ -> getLine
Why not:
const getLine

Como te llamas?
Hola, asd!

El operador (>>=) se ocupa de tomar el valor del lado izquierdo y combinarlo con la función del lado derecho para producir un valor nuevo

In [ ]:
do
    putStrLn "Como te llamas?"
    name <- getLine
    putStrLn ( "Hola " ++ name ++ "!" )

 - Código imperativo

### Las Leyes de las Mónadas

```Haskell
-- Identidad por la izquierda
return x >>= f = f x

-- Identidad por la derecha
m >>= return = m

-- Asociatividad
(m >>= f) >>= g = m >>= (x -> f x >>= g)
```

```Haskell
-- Operador Composición
(>=>) :: Monad m => ( a -> m b ) -> ( b -> m c ) -> a -> m c
(m >=> n ) x = do
    y <- m x
    n y
```

```Haskell
-- Identidad por la izquierda
return >=> f = f
-- Identidad por la derecha
f >=> return = f
-- Asociatividad
(f >=> g) >=> h = f >=> (g >=> h)
```

| Mónada     | Semántica imperativa  |
|------------| ----------------------|
| Maybe      | Excepción anónima     |
| Error      | Excepción con descripción |
| State      | Estado global |
| IO         | Entrada y Salida |
| [] (lista) | Indeterminismo |
| Reader     | Entorno |
| Writer     | Logger| 

Maybe

```Haskell
data Maybe a = Nothing | Just a

instance Monad Maybe where
    return = Just
    Nothing >>= f = Nothing
    (Just x) >>= f = f x
```


Útil para encadenar computaciones que pueden devolver Nothing como resultado

Either

In [10]:
data Either a b = Left a | Right b

Lista

```Haskell
instance Monad [] where
    m >>= f = concat (map f m)
    return x = [x]
```

In [16]:
do
    x <- [1 .. 10]
    y <- [2, 3, 5, 7]
    return (x * y)

[2,3,5,7,4,6,10,14,6,9,15,21,8,12,20,28,10,15,25,35,12,18,30,42,14,21,35,49,16,24,40,56,18,27,45,63,20,30,50,70]

In [18]:
[ x * y | x <- [1 .. 10], y <- [2, 3, 5, 7] ]

[2,3,5,7,4,6,10,14,6,9,15,21,8,12,20,28,10,15,25,35,12,18,30,42,14,21,35,49,16,24,40,56,18,27,45,63,20,30,50,70]

State

In [20]:
newtype State s a = State {runState :: (s -> (a, s))}

instance Monad ( State s ) where
    return x = State $ \s -> ( x, s )
    ( State h ) >>= f = State $ \s -> let ( a, newState ) = h s
                                          ( State g ) = f a
                                          in g newState

In [53]:
import Control.Monad

type Stack = [Int]

pop :: State Stack Int
pop = State $ \(x:xs) -> (x, xs)

push :: Int -> State Stack ()
push a = State $ \xs -> ((), a : xs)

do
    push 3
    pop

No instance for (Show (State main:Stack Int)) arising from a use of `print'
Possible fix: add an instance declaration for (Show (State main:Stack Int))
In a stmt of an interactive GHCi command: print it